In [1]:
import numpy as np

class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)

    def _calculate_gini(self, y):
        classes, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        gini = 1 - np.sum(probabilities ** 2)
        return gini

    def _find_best_split(self, X, y):
        m, n = X.shape
        if m <= 1:
            return None, None  # No split if there's only one sample

        current_gini = self._calculate_gini(y)
        best_gini = 1.0
        best_feature, best_threshold = None, None

        for feature in range(n):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask

                if np.sum(left_mask) > 0 and np.sum(right_mask) > 0:
                    left_gini = self._calculate_gini(y[left_mask])
                    right_gini = self._calculate_gini(y[right_mask])

                    weighted_gini = (np.sum(left_mask) / m) * left_gini + (np.sum(right_mask) / m) * right_gini

                    if weighted_gini < best_gini:
                        best_gini = weighted_gini
                        best_feature = feature
                        best_threshold = threshold

        return best_feature, best_threshold

    def _build_tree(self, X, y, depth):
        unique_classes, counts = np.unique(y, return_counts=True)

        # If there's only one class or max depth is reached, create a leaf node
        if len(unique_classes) == 1 or (self.max_depth is not None and depth == self.max_depth):
            return {'class': unique_classes[np.argmax(counts)]}

        # Find the best split
        best_feature, best_threshold = self._find_best_split(X, y)

        # If no split improves purity, create a leaf node
        if best_feature is None:
            return {'class': unique_classes[np.argmax(counts)]}

        # Split the data
        left_mask = X[:, best_feature] <= best_threshold
        right_mask = ~left_mask

        left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        return {'feature': best_feature, 'threshold': best_threshold,
                'left': left_subtree, 'right': right_subtree}

    def _predict_instance(self, instance, tree):
        if 'class' in tree:
            return tree['class']
        else:
            if instance[tree['feature']] <= tree['threshold']:
                return self._predict_instance(instance, tree['left'])
            else:
                return self._predict_instance(instance, tree['right'])

    def predict(self, X):
        predictions = []
        for instance in X:
            predictions.append(self._predict_instance(instance, self.tree))
        return np.array(predictions)

# Example usage:
# Assuming X_train and y_train are your training data
X_train = np.array([[1, 2], [2, 3], [3, 4], [4, 5]])
y_train = np.array([0, 0, 1, 1])

# Create and fit the decision tree
dt = DecisionTree(max_depth=2)
dt.fit(X_train, y_train)

# Assuming X_test is your test data
X_test = np.array([[2, 3], [3, 4]])
predictions = dt.predict(X_test)

print("Predictions:", predictions)


Predictions: [0 1]


A Decision Tree is a versatile supervised machine learning algorithm that can be used for both classification and regression tasks. It works by recursively partitioning the input space (feature space) into regions and assigning a label or value to each region. The decision tree is constructed based on the features of the training data, and it mimics a tree-like structure with nodes, branches, and leaves.

### Basic Concepts of Decision Trees:

1. **Node:**
   - A node represents a decision or a test on a specific feature.

2. **Root Node:**
   - The topmost node in the tree is called the root node, and it represents the feature that provides the best split for the entire dataset.

3. **Internal Node:**
   - Nodes in the tree, other than the root node, are called internal nodes. Internal nodes represent decisions or tests on specific features.

4. **Leaf (Terminal) Node:**
   - The end nodes of the tree are called leaves or terminal nodes. Leaves provide the final prediction or output.

5. **Splitting:**
   - The process of dividing the dataset into subsets based on a feature test is called splitting.

6. **Feature Test:**
   - Each internal node performs a test on one feature, and the outcome determines which branch (left or right) to follow.

7. **Decision Criteria:**
   - The decision criteria for a node involve selecting the feature and threshold that optimally splits the data, often based on impurity measures like Gini impurity or information gain.

### Construction of Decision Trees:

1. **Selecting the Best Split:**
   - At each internal node, the algorithm selects the feature and threshold that provides the best split, maximizing information gain (for classification) or reducing variance (for regression).

2. **Recursive Partitioning:**
   - The dataset is recursively partitioned based on the selected features and thresholds until a stopping condition is met (e.g., maximum depth, minimum samples per leaf).

3. **Stopping Conditions:**
   - Stopping conditions prevent the tree from growing too complex and overfitting the training data. Common stopping conditions include a maximum depth for the tree or a minimum number of samples required to split a node.

### Decision Tree Types:

1. **Classification Trees:**
   - Used for predicting categorical labels or classes.

2. **Regression Trees:**
   - Used for predicting continuous numerical values.

### Use Cases:

- **Classification:**
  - Predicting whether an email is spam or not.

- **Regression:**
  - Predicting the price of a house based on its features.

- **Decision Support Systems:**
  - Helping in decision-making by providing a transparent and interpretable model.

- **Data Exploration:**
  - Understanding feature importance and relationships in the data.

### Advantages and Considerations:

- **Advantages:**
  - Easy to understand and interpret.
  - Requires minimal data preprocessing.
  - Nonlinear relationships can be captured.

- **Considerations:**
  - Prone to overfitting, especially with deep trees.
  - Sensitive to noisy data.
  - Biased towards features with more levels.

### Ensemble Methods:

- **Random Forests:**
  - An ensemble of decision trees that can improve generalization performance by reducing overfitting.

- **Gradient Boosted Trees:**
  - Builds decision trees sequentially, with each tree correcting the errors of the previous one.

Decision Trees are powerful tools for both predictive modeling and data exploration. While they have certain limitations, they serve as a foundation for more advanced ensemble methods that mitigate some of these challenges. Their simplicity and interpretability make them valuable in various domains, especially when transparency and understanding of the model are crucial.

In [3]:
import numpy as np

class RandomForest:
    def __init__(self, n_trees=10, max_depth=None, bootstrap_ratio=0.8):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.bootstrap_ratio = bootstrap_ratio
        self.trees = []

    def fit(self, X, y):
        for _ in range(self.n_trees):
            # Bootstrap sample
            indices = np.random.choice(X.shape[0], size=int(self.bootstrap_ratio * X.shape[0]), replace=True)
            X_bootstrap, y_bootstrap = X[indices], y[indices]

            # Train a decision tree
            tree = DecisionTree(max_depth=self.max_depth)
            tree.fit(X_bootstrap, y_bootstrap)
            self.trees.append(tree)

    def predict(self, X):
        # Make predictions using all trees and combine them
        predictions = np.array([tree.predict(X) for tree in self.trees])
        # Use majority voting for classification
        return np.mean(predictions, axis=0) > 0.5

# Decision Tree class (similar to the previous implementation)
#class DecisionTree:
    # ... (Same Decision Tree implementation as provided earlier)

# Example usage:
# Assuming X_train and y_train are your training data
X_train = np.array([[1, 2], [2, 3], [3, 4], [4, 5], [5, 6]])
y_train = np.array([0, 0, 1, 1, 1])

# Create and fit the Random Forest
rf = RandomForest(n_trees=5, max_depth=2)
rf.fit(X_train, y_train)

# Assuming X_test is your test data
X_test = np.array([[2, 3], [4, 5]])
predictions = rf.predict(X_test)

print("Predictions:", predictions)


Predictions: [ True  True]


Random Forest is an ensemble learning algorithm that builds a collection of decision trees and combines their predictions to improve overall performance and generalization. It is widely used for both classification and regression tasks due to its effectiveness and robustness. The key idea behind Random Forest is to introduce randomness during both the training phase and the prediction phase to create diverse and uncorrelated trees. Here are the main concepts of Random Forest:

### Key Concepts:

1. **Ensemble Learning:**
   - Random Forest belongs to the ensemble learning family, where multiple models are trained and their predictions are combined to make a final prediction.

2. **Decision Trees:**
   - The base model used in a Random Forest is the decision tree. Decision trees are simple yet powerful models that recursively split the feature space based on feature values.

3. **Bootstrap Sampling:**
   - During the training phase, each tree in the Random Forest is trained on a different subset of the data created through bootstrap sampling (sampling with replacement). This introduces diversity among the trees.

4. **Feature Randomness:**
   - At each node of a decision tree, only a random subset of features is considered for splitting. This ensures that each tree specializes in different aspects of the data.

5. **Voting (Classification) or Averaging (Regression):**
   - For classification tasks, the final prediction is often determined by a majority vote among the individual tree predictions. For regression tasks, predictions are averaged.

6. **Out-of-Bag (OOB) Error:**
   - Since each tree is trained on a different subset of the data, there are data points that are not included in the training of certain trees. These out-of-bag samples can be used to estimate the performance of the model without the need for a separate validation set.

### Training Process:

1. **Bootstrap Sampling:**
   - For each tree in the forest, a random subset of the training data is sampled with replacement. This results in multiple bootstrap samples.

2. **Tree Construction:**
   - A decision tree is constructed for each bootstrap sample, considering only a random subset of features at each split.

3. **Ensemble Building:**
   - The collection of decision trees forms the Random Forest ensemble.

### Prediction Process:

1. **Voting or Averaging:**
   - For classification tasks, the class with the majority of votes among the trees is the final prediction. For regression tasks, the average of the individual tree predictions is taken.

### Advantages:

1. **High Accuracy:**
   - Random Forests generally provide high accuracy and robustness, often outperforming individual decision trees.

2. **Reduced Overfitting:**
   - The ensemble approach reduces overfitting by averaging out the idiosyncrasies of individual trees.

3. **Feature Importance:**
   - Random Forests can provide information about feature importance, helping in feature selection.

4. **Robust to Noisy Data:**
   - Random Forests are less sensitive to noisy data compared to individual decision trees.

### Use Cases:

- **Classification:**
  - Image recognition, spam detection, etc.

- **Regression:**
  - Predicting house prices, demand forecasting, etc.

- **Feature Importance Analysis:**
  - Identifying critical features in a dataset.

### Considerations:

- **Computational Cost:**
  - Training multiple trees can be computationally expensive, but the process can be parallelized.

- **Interpretability:**
  - While decision trees are interpretable, the ensemble nature of Random Forests makes them less interpretable.

Random Forests are a powerful and versatile algorithm widely used in practice. They are suitable for a variety of tasks and are known for their robustness and ability to handle complex relationships in the data.